# Deep Dream

In [1]:
import tensorflow.keras as keras
keras.__version__

'2.2.4-tf'

In [2]:
import tensorflow as tf
# workaround needed to support K.gradients()
# Source: https://github.com/tensorflow/tensorflow/issues/34235
tf.compat.v1.disable_eager_execution()  

## Load pretrained model

In [3]:
from tensorflow.keras.applications import inception_v3
from tensorflow.keras import backend as K

In [4]:
# we'll only run the net in inference mode
K.set_learning_phase(0)

## Define loss to be maximized

In [13]:
def build_loss(model, layer_contributions):
    print(layer_contributions)

    layer_dict = dict([(layer.name, layer) for layer in model.layers])

    _loss = K.variable(0.)
    for layer_name, coeff in layer_contributions.items():
        activation = layer_dict[layer_name].output

        # Add L2 loss of features to the loss, avoid border pixels
        scaling = K.prod(K.cast(K.shape(activation), 'float32'))
        _loss = _loss + coeff * K.sum(K.square(activation[:, 2: -2, 2: -2, :])) / scaling
    #     K.update_add(loss, coeff * K.sum(K.square(activation[:, 2:-2, 2:-2, :])) / scaling)
    
    return _loss


## Gradient ascent

In [14]:
def build_fetch_fn(model, loss):

    dream = model.input
    
    grads = K.gradients(loss, dream)[0]

    # normalize gradients, important trick
    grads /= K.maximum(K.mean(K.max(grads)), 1e-7)

    # Keras function to retreive value of loss and gradients, given an input image
    outputs = [loss, grads]
    fetch_loss_and_grads = K.function([dream], outputs)
    
    return fetch_loss_and_grads

def eval_loss_and_grads(x, fetch_fn):
    outs = fetch_fn([x])
    return outs[0], outs[1]  # loss_values, grad_values

def gradient_ascent(x, iterations, step, fetch_fn, max_loss=None):
    for i in range(iterations):
        loss_value, grad_values = eval_loss_and_grads(x, fetch_fn)
        if max_loss is not None and loss_value >= max_loss:
            break
        print("... Loss value at {}: {}".format(i, loss_value))
        x += step * grad_values
    return x

## Running gradient ascent over successive scales

In [15]:
# helper functions
import numpy as np
import scipy
from tensorflow.keras.preprocessing import image

def resize_img(img, size):
    img = np.copy(img)
    factors = (1,
              float(size[0]) / img.shape[1],
              float(size[1]) / img.shape[2],
              1)
    return scipy.ndimage.zoom(img, factors, order=1)

def save_img(img, fname):
    pil_img = deprocess_image(np.copy(img))
    scipy.misc.imsave(fname, pil_img)
    
def preprocess_image(image_path):
    img = image.load_img(image_path)
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    return inception_v3.preprocess_input(img)

def deprocess_image(x):
    if K.image_data_format() == 'channels_first':
        x = x.reshape((3, x.shape[2], x.shape[3]))
        x = x.transpose((1, 2, 0))
    else:
        x = x.reshape((x.shape[1], x.shape[2], 3))
    x /= 2.
    x += 0.5
    x *= 255.
    return np.clip(x, 0, 255).astype('uint8')

In [18]:
def deepdream(model,
              base_image_path, 
              image_prefix='dream', 
              step=0.05, 
              num_octaves=3,
              octave_scale=1.4,
              iterations=15, 
              max_loss=15., 
              layer_contributions=None):
    """
    :param model: Keras model object
    :param base_image_path: Path to source image
    :param image_prefix: Prefix to use when writing file, use to avoid overwriting files
    :param step: Gradient ascent step size. Higher value results in more visible artifacts (TODO correct?)
    :param num_octaves: Number of scales at which to run gradient ascent
    :param octave_scale: Size ratio between scales
    :param iterations: Number of ascent steps to run
    :param max_loss: Interrupt gradient ascent if loss grows too large
    :param layer_contributions: dict that defines how much each named layer contributes to the outcome
    """

    assert layer_contributions is not None  # TODO better default handling
    
    loss = build_loss(model, layer_contributions)
    fetch_fn = build_fetch_fn(model, loss)
    
    img = preprocess_image(base_image_path)

    original_shape = img.shape[1:3]
    successive_shapes = [original_shape]
    for i in range(1, num_octaves):
        shape = tuple([int(dim / (octave_scale ** i)) for dim in original_shape])
        successive_shapes.append(shape)

    # reverse so the shapes are in increasing order
    successive_shapes = successive_shapes[::-1]

    original_img = np.copy(img)
    shrunk_original_img = resize_img(img, successive_shapes[0])

    for shape in successive_shapes:
        print("Processing shape:", shape)

        # Scale up dream image
        img = resize_img(img, shape)

        img = gradient_ascent(img, iterations, step, fetch_fn, max_loss)

        # scale up smaller original, will be blurry
        upscaled_shrunk_original_img = resize_img(shrunk_original_img, shape)

        # non-blurry version of original
        same_size_original = resize_img(original_img, shape)

        # Re-inject details that were lost when scaling up
        lost_detail = same_size_original - upscaled_shrunk_original_img
        img += lost_detail

        # prepare for next iteration
        shrunk_original_img = resize_img(original_img, shape)

        #save_img(img, fname='{}_dream_at_scale_{}.png'.format(image_prefix, str(shape)))

    save_img(img, fname='{}_final.png'.format(image_prefix))

In [22]:
steps = (0.12, 0.16)

model = inception_v3.InceptionV3(weights='imagenet', include_top=False)

layer_contributions = {
    'mixed2': 0.2,
    'mixed3': 3.,
    'mixed4': 2.,
    'mixed5': 5
}

for i in range(6):
    
    step = np.random.choice(steps)
    
    layer_contributions = {
        'mixed2': 5*np.random.random(),
        'mixed3': 5*np.random.random(),
        'mixed4': 5*np.random.random(),
        'mixed5': 5*np.random.random()
    }
    

    print("=== Processing step {} ===".format(step))

    base_image_path = '/home/fletsch/Downloads/elphi.jpg'
    image_prefix = 'elphi{}'.format(str(i))
    
    deepdream(model, base_image_path, image_prefix, step=step, layer_contributions=layer_contributions)
    
print("===[ D O N E ]===")

    

=== Processing step 0.16 ===
{'mixed2': 4.644038166163908, 'mixed3': 1.4645637162154572, 'mixed4': 4.761898256949799, 'mixed5': 1.0299101539147988}
Tensor("add_83:0", shape=(), dtype=float32)
Tensor("input_7:0", shape=(None, None, None, 3), dtype=float32)
Processing shape: (306, 306)
... Loss value at 0: 2.7147483825683594
... Loss value at 1: 3.1217494010925293
... Loss value at 2: 3.5991928577423096
... Loss value at 3: 4.04488468170166
... Loss value at 4: 4.641900062561035
... Loss value at 5: 5.171279430389404
... Loss value at 6: 5.674116611480713
... Loss value at 7: 5.949350357055664
... Loss value at 8: 6.474755764007568
... Loss value at 9: 6.835021495819092
... Loss value at 10: 7.526459217071533
... Loss value at 11: 8.205368041992188
... Loss value at 12: 8.462352752685547
... Loss value at 13: 9.185357093811035
... Loss value at 14: 9.333290100097656
Processing shape: (428, 428)
... Loss value at 0: 3.8874621391296387
... Loss value at 1: 4.979705333709717
... Loss value 

/home/fletsch/miniconda3/envs/avatar/lib/python3.6/site-packages/ipykernel_launcher.py:16: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
  app.launch_new_instance()


=== Processing step 0.12 ===
{'mixed2': 2.98073362189313, 'mixed3': 1.8757607034648927, 'mixed4': 3.1168481889130355, 'mixed5': 4.152047937392079}
Tensor("add_87:0", shape=(), dtype=float32)
Tensor("input_7:0", shape=(None, None, None, 3), dtype=float32)
Processing shape: (306, 306)
... Loss value at 0: 2.49973201751709
... Loss value at 1: 2.8304991722106934
... Loss value at 2: 3.1967639923095703
... Loss value at 3: 3.6184191703796387
... Loss value at 4: 3.9790921211242676
... Loss value at 5: 4.315735816955566
... Loss value at 6: 4.653691291809082
... Loss value at 7: 5.224588394165039
... Loss value at 8: 5.593934535980225
... Loss value at 9: 5.8247528076171875
... Loss value at 10: 6.145998954772949
... Loss value at 11: 6.671083450317383
... Loss value at 12: 6.881475448608398
... Loss value at 13: 7.370504856109619
... Loss value at 14: 7.544546604156494
Processing shape: (428, 428)
... Loss value at 0: 3.3834781646728516
... Loss value at 1: 4.245903491973877
... Loss value

... Loss value at 10: 6.934264659881592
... Loss value at 11: 7.1869611740112305
... Loss value at 12: 7.617256164550781
... Loss value at 13: 8.218668937683105
... Loss value at 14: 8.449460983276367
=== Processing step 0.12 ===
{'mixed2': 0.4456735914193599, 'mixed3': 1.02597929938592, 'mixed4': 3.944481459834053, 'mixed5': 4.710008749537932}
Tensor("add_103:0", shape=(), dtype=float32)
Tensor("input_7:0", shape=(None, None, None, 3), dtype=float32)
Processing shape: (306, 306)
... Loss value at 0: 1.5928683280944824
... Loss value at 1: 1.852832555770874
... Loss value at 2: 2.1331591606140137
... Loss value at 3: 2.391953706741333
... Loss value at 4: 2.77587628364563
... Loss value at 5: 3.0517797470092773
... Loss value at 6: 3.2750232219696045
... Loss value at 7: 3.6427273750305176
... Loss value at 8: 4.0242462158203125
... Loss value at 9: 4.332034111022949
... Loss value at 10: 4.695204257965088
... Loss value at 11: 4.873055458068848
... Loss value at 12: 5.264019012451172


## Create a GIF

Use the following imagemagick command on your command line to produce an animated GIF out of the single frames. 
Note that `-morph <num>` is optional, it caused intermediate morphed frames between the single frames, to make the changes appear more smooth.

```
convert -delay 20 -loop 0 -morph 3 *png plant.gif

```